In [6]:
# Imports
import torch
import torch.nn as nn 
from torchvision import transforms
import numpy as np
from  matplotlib import pyplot as plt
import torch.nn.functional as F
import time
from utils import get_data, show_batch
import itertools

In [2]:
# Class for storing things such as learning rate, image size...
class Args:
    def __init__(self):
        self.lr = 2e-4
        self.epochs = 200
        self.b1 = 0.5
        self.b2 = 0.999
        self.latent_dim = 100
        self.img_size = 64
        self.pixels = int(self.img_size ** 2)
        self.channels = 3
        self.img_tuple = (self.channels, self.img_size, self.img_size)
        self.batch_size = 32
        self.g_fmap_size = 64 
        self.d_fmap_size = 64
        self.d_loss_threshold = 0.5
        self.n_res_blocks = 9

In [3]:
# Loading the images.
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
args = Args()
transform = transforms.Compose([transforms.Resize(args.img_size),
                                transforms.CenterCrop(args.img_size),
                                transforms.ToTensor(),
                                transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])
paintings, photos, painting_loader, photo_loader = get_data(transform=transform, batch_size=args.batch_size)

In [5]:
# Defining the Generator. 
class ResidualBlock(nn.Module):
    def __init__(self, in_features):
        super().__init__()
        self.pad1 = nn.ReflectionPad1d(1)
        self.pad2 = nn.ReflectionPad1d(in_features)
        self.conv1 = nn.Conv2d(in_features, in_features, 3)
        self.conv2 = nn.Conv2(in_features, in_features, 3)
        self.norm1 = nn.InstanceNorm2d(in_features)
        self.norm2 = nn.InstanceNorm2d(in_features)
    
    def block(self, x):
        x = self.pad1(x)
        x = self.conv1(x)
        x = self.norm1(x)
        x = x.relu()
        x = self.pad2(x)
        x = self.conv2(x)
        return self.norm2(x)
    
    def forward(self, x):
        return x + self.block(x)
    
class Generator(nn.Module):
    def __init__(self, num_residual_block):
        super().__init__()
        self.out_features = 64
        self.in_features = self.out_features
        self.num_residual_blocks = num_residual_block
        self.__compile_model()
        
    # First convolutional block. 
    def __conv1(self):
        self.model += [
            nn.ReflectionPad2(args.channels),
            nn.Conv2d(args.channels, self.out_features, 7),
            nn.InstanceNorm2d(self.out_features),
            nn.ReLU(inplace=True)
            ]
                       
    # Downsampling
    def __downsample(self):
        for _ in range(2):
            self.out_features *= 2  
            self.model += [
                nn.Conv2d(self.in_features, self.out_features, 3, stride=2, padding=1),
                nn.InstanceNorm2d(self.out_features),
                nn.ReLU(inplace=True)
            ]
        self.in_features = self.out_features

    def __residual_blocks(self):
        for _ in range(self.num_residual_blocks):
            model += [ResidualBlock(self.out_features)]

    # Upsampling 
    def __upsample(self):
        for _ in range(2):
            self.out_features //= 2
            self.model += [
                nn.Upsample(scale_factor=2),
                nn.Conv2d(self.in_features, self.out_features, 3, stride=1, padding=1),
                nn.ReLU(inplace=True)
            ]
        self.in_features = self.out_features

    # Output layer
    def __output(self):
        model += [
            nn.ReflectionPad2d(args.channels),
            nn.Conv2d(self.out_features, args.channels, 7),
            nn.Tanh()
        ]
    
    # Compiles the model
    def __compile_model(self):
        self.model = []
        self.__conv1()
        self.__downsample()
        self.__residual_blocks()
        self.__upsample()
        self.__output()
        self.model = nn.Sequential(*self.model)
    
    def forward(self, x):
        return self.model(x)


In [4]:
# Defining the discriminator.
class Discriminator(nn.Module):
    def __init__(self):
        super().__init__()
        self.output_shape = (1, args.img_size // 2 ** 4, args.img_size // 2 ** 4)
        self.__compile_model()
    
    def __discriminator_block(self, in_filters, out_filters, normalize=True):
        layers = [nn.Conv2d(in_filters, out_filters, 4, stride=2, padding=1)]
        if normalize:
            layers.append(nn.InstanceNorm2d(out_filters))
        layers.append(nn.LeakyReLU(0.2, inplace=True))
        return layers 

    def __compile_model(self):
        self.model = nn.Sequential[
            self.__discriminator_block(args.channels, 64,  normalize=False),
            self.__discriminator_block(64, 128),
            self.__discriminator_block(128, 256),
            self.__discriminator_block(256, 512),
            nn.ZeroPad2d((1, 0, 1, 0)),
            nn.Conv2d(512, 1, 4, padding=1)
        ]
    
    def forward(self, x):
        return self.model(x)

In [ ]:
# Initializes weights to a Gaussian distribution. 
def init_weights(m):
    classname = m.__class__.__name__
    if classname.find('Conv') != -1:
        nn.init.normal_(m.weight.data, 0.0, 0.02)
        if hasattr(m, 'bias') and m.bias is not None:
            nn.init.constant_(m.bias.data, 0.0)
    elif classname.find('BatchNorm2d') != -1:
        nn.init.normal_(m.weight.data, 1.0, 0.02)
        nn.init.constant_(m.bias.data, 0.0)

In [ ]:
# Initalizing the networks. 
gen_ab = Generator(args.img_tuple, args.n_res_blocks).to(device)
gen_ba = Generator(args.img_tuple, args.n_res_blocks).to(device)
disc_a = Discriminator(args.img_tuple).to(device)
disc_b = Discriminator(args.img_tuple).to(device)
loss_gen = nn.MSELoss()
loss_cycle = nn.L1Loss()
loss_identity = nn.L1Loss()

gen_ab.apply(init_weights)
gen_ba.apply(init_weights)
disc_a.apply(init_weights)
disc_b.apply(init_weights)

In [ ]:
# Setting up optimizers. 
optim_g = torch.optim.Adam(
    itertools.chain(gen_ab.parameters(), gen_ba.parameters()), lr=args.lr, betas=(args.b1, args.b2)
    )
optim_d_a = torch.optim.Adam(
    disc_a.parameters(), lr=args.lr, betas=(args.b1, args.b2)
)
optim_d_b = torch.optim.Adam(
    disc_b.parameters(), lr=args.lr, betas=(args.b1, args.b2)
)